Aim: To calculate the total travelers per week from the first date of travel of individual airlines

In [70]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [95]:
#importing data
total_trav = pd.read_csv("df_all.csv")
#obtain shape
total_trav.shape

(996, 4)

In [96]:
total_trav.head(3)

,Travelers,Day,Month,Year
0,2882915.0,28,Nov,2019
1,2648268.0,27,Nov,2019
2,1968137.0,26,Nov,2019


In [97]:
total_trav.tail(3)

,Travelers,Day,Month,Year
993,1327289.0,3,Jan,2021
994,1192881.0,2,Jan,2021
995,805990.0,1,Jan,2021


Convert the Day, Month, and Year columns to one date variable

In [98]:
#changing Month values to numbers
total_trav = total_trav.replace({'Month': {"Jan": 1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,
                                             'Oct':10,'Nov':11,'Dec':12}})
#converting Day, Month, and Year columns to string and combining the three columns 
total_trav['date'] = total_trav.Day.astype(str) + '-' + total_trav.Month.astype(str) + '-' + total_trav.Year.astype(str)
total_trav.head()


,Travelers,Day,Month,Year,date
0,2882915.0,28,11,2019,28-11-2019
1,2648268.0,27,11,2019,27-11-2019
2,1968137.0,26,11,2019,26-11-2019
3,1591158.0,25,11,2019,25-11-2019
4,2624250.0,24,11,2019,24-11-2019


In [99]:
#converting date to a date variable
total_trav['date'] = pd.to_datetime(total_trav['date'])
#to count number of events create a new column and assign 1 to each row
# post_long['event_num'] = 1
total_trav.date.dtype

dtype('<M8[ns]')

Randomly assign airlines to a new column. This is done just for analyses purposes

In [100]:
#list of airports
#for consistency let's set a seed
np.random.seed(123)
airlines = ['AS','G4','AA','XP','MX','DL','2D','F9','HA','B6','Test']
total_trav['airlines'] = np.random.choice(list(airlines), len(total_trav))

In [118]:
total_trav.head(10)

,Travelers,Day,Month,Year,date,airlines
0,2882915.0,28,11,2019,2019-11-28,AA
1,2648268.0,27,11,2019,2019-11-27,AA
2,1968137.0,26,11,2019,2019-11-26,2D
3,1591158.0,25,11,2019,2019-11-25,G4
4,2624250.0,24,11,2019,2019-11-24,XP
5,2435170.0,23,11,2019,2019-11-23,Test
6,2254188.0,22,11,2019,2019-11-22,B6
7,2321546.0,21,11,2019,2019-11-21,2D
8,2194291.0,20,11,2019,2019-11-20,G4
9,2550459.0,19,11,2019,2019-11-19,AS


For each airline, calculate the total travelers each week from the first date of travel of the individual airlines

In [102]:
#We do not anticipate missing airlines on our list since we inputed that, but let's just check for missing airlines
filter_airlines = total_trav['airlines'].isnull()
sum(filter_airlines)


0

In [104]:
#drop rows where airline is 'Test'. 
filter_airlines1 = total_trav['airlines']=='Test'
total_trav_filtered = total_trav[filter_airlines1 == False]
total_trav_filtered.head()

,Travelers,Day,Month,Year,date,airlines
0,2882915.0,28,11,2019,2019-11-28,AA
1,2648268.0,27,11,2019,2019-11-27,AA
2,1968137.0,26,11,2019,2019-11-26,2D
3,1591158.0,25,11,2019,2019-11-25,G4
4,2624250.0,24,11,2019,2019-11-24,XP


In [105]:
total_trav_filtered.shape

(919, 6)

In [110]:
#creating a list of airlines and removing the last item(test)
airline_list = airlines[0:10]
print(airline_list)

['AS', 'G4', 'AA', 'XP', 'MX', 'DL', '2D', 'F9', 'HA', 'B6']


In [119]:
def wkly_trav_per_airline(df,airline):
    #grouping data by airlines and date
    df_trav = df.groupby(['airlines','date']).Travelers.sum().to_frame(name = 'Travelers').reset_index()
    #subsetting data to show only the airline that is being analyzed
    uniq_airline = df_trav[df_trav['airlines'] == airline]
    #grouping data by week for the specified airline
    airline_week = uniq_airline.groupby(pd.Grouper(key="date", freq="W")).sum() 
    #add airline name
    airline_week['airlines'] = airline
    
    airline_week_clean1 = airline_week.reset_index()
    airline_week_clean1 = airline_week_clean1.rename(columns={"index":"Week"})
    airline_week_clean1['Week'] = airline_week_clean1.index
    
    return airline_week


In [ ]:
#calling the wkly_trav_per_airline function in a for loop for the individual airlines
airline_trav_list = []
  
for airlin in airline_list:
    airline_data = wkly_trav_per_airline(total_trav_filtered,airlin)
    airline_trav_list.append(airline_data)
    all_output = pd.concat(airline_trav_list)

In [126]:
all_output= all_output.reset_index()